In [15]:
import json
import os
import copy

In [16]:
data_dir = '/opt/ml/input/data'

In [17]:
# train_all.json 정보 불러오기
with open(os.path.join(data_dir, 'train_all.json'), 'r', encoding='UTF-8') as train_json:
    train_data = json.load(train_json)
    
    info = train_data['info']
    licenses = train_data['licenses']
    categories = train_data['categories']
    
    train_images = train_data['images']
    train_annotations = train_data['annotations']

In [18]:
# data.json 정보 불러오기
with open(os.path.join(data_dir, 'batch_03/data.json'), 'r', encoding='UTF-8') as leaked_json:
    leaked_data = json.load(leaked_json)
    
    leaked_images = leaked_data['images']
    leaked_annotations = leaked_data['annotations']

In [19]:
# leaked_data.txt는 구글 드라이브에 업로드해놨습니다.
with open(os.path.join(data_dir, 'leaked_data.txt'), 'r') as f:
    leaked_files = f.readlines()
    leaked_files = list(map(lambda x : x.strip(), leaked_files))

In [20]:
new_images = copy.deepcopy(train_images)
new_annotations = copy.deepcopy(train_annotations)

# 추가 시 사용할 annotation_id
annotation_id = train_annotations[-1]['id'] + 1

# 추가 시 사용할 image_id
image_id = train_images[-1]['id'] + 1

# 유출 된 annotation 합치기
for img in leaked_images:
    if img['file_name'] in leaked_files:
        flag = False
        for ann in leaked_annotations:
            if ann['category_id'] == 0:
                flag = True
                break
        if flag: continue

        for ann in leaked_annotations:
            if ann['image_id'] == img['id']:
                new_annotations.append(copy.deepcopy(ann))
                # 추가된 annotation의 id와 image_id 수정
                new_annotations[-1]['id'] = annotation_id
                new_annotations[-1]['image_id'] = image_id
                annotation_id += 1

        new_images.append(copy.deepcopy(img))
        #추가된 images의 id 수정
        new_images[-1]['id'] = image_id
        image_id += 1

In [ ]:
# input/data 경로에 new_train_all.json으로 저장
with open('/opt/ml/input/data/new_train_all_img_excluded.json', 'w', encoding='UTF-8') as new_json:
        json.dump({ 'info': info, 'licenses': licenses, 'images': new_images, 
            'annotations': new_annotations, 'categories': categories}, new_json, indent=2)